In [1]:

import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.2


In [2]:
from azureml.core import Workspace
#ws=Workspace.get(name='zAML',subscription_id='my_subscription_id')


In [ ]:
# Create the configuration file.
#ws.write_config()

# Use this code to load the workspace from 
# other scripts and notebooks in this directory.
# ws = Workspace.from_config()

In [4]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      #'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /home/nbuser/library/aml_config/config.json
Workspace name: zAML
Azure region: westeurope
Resource group: AMLrg


In [5]:

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [6]:

from azureml.core import Datastore

ds = ws.get_default_datastore()
print("Using the default datastore for training data: ")
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

Using the default datastore for training data: 
workspaceblobstore AzureBlob zaml5744793264 azureml-blobstore-9d836814-f895-4a65-af78-59f755a38c85


In [7]:
 
from azureml.core import Datastore
# only need to do it once
ds2 = Datastore.register_azure_file_share(workspace=ws, 
                                         datastore_name='CTscands', 
                                         file_share_name='bloodcell',
                                         account_name='zbatchaistorage', 
                                         account_key='MmkzeKphqtgv83Z3DfieWNF40bdj53FQ7WebO2zPW73Jh3YOxOgGuAkj/gi0Bo/4K+awcWtlGvpDE9tIQvKYYw==',
                                         create_if_not_exists=False)
                                         

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-01-09T13:21:06.234000+00:00', 'creationTime': '2019-01-08T10:23:55.033355+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-01-08T10:25:35.793472+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [20]:
import os

project_folder = './keras-ctscan-exp'
os.makedirs(project_folder, exist_ok=True)

In [35]:
import shutil

#shutil.copy('keras_cnn_dicom.py', project_folder)
shutil.copy('keras_cnn_pydicom.py', project_folder)


'./keras-ctscan-exp/keras_cnn_pydicom.py'

In [36]:
cat ./keras-ctscan-exp/keras_cnn_pydicom.py

from __future__ import print_function
import argparse
import pydicom
from matplotlib import pyplot, cm
import os
import sys
import numpy as np
import pandas as pd
import scipy
import keras
from keras.models import Sequential
from keras.layers import AveragePooling2D , Convolution2D , Flatten ,Dense, MaxPooling2D, Conv2D
from keras.preprocessing import utils
from keras.preprocessing.image import ImageDataGenerator


def get_data(dicom_dir):
    #resize the image to desired resolution
    #print("dicom_dir",os.listdir(dicom_dir), dicom_dir)
    xsize = 256; ysize = 256
    
    data = np.zeros((xsize, ysize, 100))
    #print("dicom_dir",os.listdir(dicom_dir), dicom_dir)
    for i, s in enumerate(os.listdir(dicom_dir)):
    
        img = np.array(pydicom.read_file(dicom_dir+ s).pixel_array)
        xscale = xsize/img.shape[0]
        yscale = ysize/img.shape[1]
        data[:,:,i] = scipy.ndimage.interpolation.zoom(img, [xscale, yscale])
    #returning a numpy array of shape 100,256,256


In [37]:
from azureml.core import Experiment

experiment_name = 'keras-tf-exp'
experiment = Experiment(ws, name=experiment_name)

In [15]:

ds2.path()

$AZUREML_DATAREFERENCE_ctscands

In [38]:
from azureml.train.dnn import TensorFlow 
script_params={
    '--data': ds2.path(),
    '--epoch': 1,
    '--save_model':'/outputs'
}

estimator = TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      entry_script='keras_cnn_pydicom.py',
                      script_params=script_params,
                      node_count=1,
                      process_count_per_node=1,
                      #distributed_backend='mpi',    
                      pip_packages=['pydicom','keras','scikit-image','scikit-learn','scipy','argparse',
                                    'opencv-contrib-python-headless','pillow','numpy', 'pandas','matplotlib'],
                      #custom_docker_base_image='zecharpy/tfgpupy3:pydicom',
                      use_gpu=True)

In [39]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: keras-tf-exp,
Id: keras-tf-exp_1547042123229,
Type: azureml.scriptrun,
Status: Queued)


In [40]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…

In [41]:
run.wait_for_completion(show_output=True)

RunId: keras-tf-exp_1547042123229


MSI: Failed to retrieve a token from 'http://localhost:25198/nb/api/nbsvc/oauth2/token' with an error of 'HTTPConnectionPool(host='localhost', port=25198): Max retries exceeded with url: /nb/api/nbsvc/oauth2/token (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9e5c36bf60>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))'. This could be caused by the MSI extension not yet fullly provisioned.



Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
/mnt/batch/tasks/shared/LS_root/jobs/zaml/azureml/keras-tf-exp_1547042123229/mounts/ctscands
check the dicom file shape should be 100, 256,256 (100, 256, 256)
keras_cnn_pydicom.py:92: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  classifier.add(Conv2D(64, 3, 3, activation = 'relu'))
keras_cnn_pydicom.py:96: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  classifier.add(Conv2D(128, 3, 3, activation = 'relu'))
keras_cnn_pydicom.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  classifier.add(Conv2D(256, 3, 3, activation = 'relu'))
keras_cnn_pydicom.py:118: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argume

100/100 [==============================] - 16s 156ms/step - loss: 0.5244 - acc: 0.7450 - val_loss: 0.0991 - val_acc: 0.9880


The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.10054397583007812 seconds

Execution Summary
RunId: keras-tf-exp_1547042123229



{'runId': 'keras-tf-exp_1547042123229',
 'target': 'gpucluster',
 'status': 'Finalizing',
 'startTimeUtc': '2019-01-09T14:02:42.167711Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '5839616d-bfcb-4ebb-8d2e-48b0332ce548'},
 'runDefinition': {'Script': 'keras_cnn_pydicom.py',
  'Arguments': ['--data',
   '$AZUREML_DATAREFERENCE_ctscands',
   '--epoch',
   '1',
   '--save_model',
   '/outputs'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 0,
  'Target': 'gpucluster',
  'DataReferences': {'ctscands': {'DataStoreName': 'ctscands',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 1,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
      {'pip